In [ ]:
                                        ###    Upload Neo4J Data   ###

import pandas as pd
from neo4j import GraphDatabase

# Cambiar por tus credenciales
URI = "bolt://neo4j:7687"
USER = "neo4j"
PASSWORD = "longerpass123"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

def cargar_datos_a_neo4j():
    # Leer archivos CSV
    proveedores = pd.read_csv('proveedor.csv', sep=';')
    telefonos = pd.read_csv('telefono.csv', sep=';')
    pedidos = pd.read_csv('op.csv', sep=';')
    productos = pd.read_csv('producto.csv', sep=';')
    detalle_op = pd.read_csv('detalle_op.csv', sep=';')

    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")

        # Crear proveedores
        for _, row in proveedores.iterrows():
            session.run("""
                MERGE (p:Proveedor {id_proveedor: $id})
                SET p.CUIT = $CUIT, p.razon_social = $razon, p.tipo_sociedad = $sociedad,
                    p.direccion = $dir, p.activo = $activo, p.habilitado = $habilitado
            """, id=row['id_proveedor'], CUIT=row['CUIT_proveedor'], razon=row['razon_social'],
                 sociedad=row['tipo_sociedad'], dir=row['direccion'],
                 activo=bool(row['activo']), habilitado=bool(row['habilitado']))

        # Crear telefonos y relaciones
        for _, row in telefonos.iterrows():
            session.run("""
                MERGE (t:Telefono {codigo_area: $area, nro_telefono: $nro})
                SET t.tipo = $tipo
                WITH t
                MATCH (p:Proveedor {id_proveedor: $id_prov})
                MERGE (p)-[:TIENE_TELEFONO]->(t)
            """, area=row['codigo_area'], nro=row['nro_telefono'], tipo=row['tipo'],
                 id_prov=row['id_proveedor'])

        # Crear pedidos y relaciones
        for _, row in pedidos.iterrows():
            session.run("""
                MERGE (o:Pedido {id_pedido: $id})
                SET o.fecha = $fecha, o.total_sin_iva = $total, o.iva = $iva
                WITH o
                MATCH (p:Proveedor {id_proveedor: $id_prov})
                MERGE (o)-[:ES_DE]->(p)
            """, id=row['id_pedido'], fecha=row['fecha'], total=row['total_sin_iva'],
                 iva=row['iva'], id_prov=row['id_proveedor'])

        # Crear productos
        for _, row in productos.iterrows():
            session.run("""
                MERGE (pr:Producto {id_producto: $id})
                SET pr.descripcion = $desc, pr.marca = $marca, pr.categoria = $cat,
                    pr.precio = $precio, pr.stock_actual = $stock, pr.stock_futuro = $futuro
            """, id=row['id_producto'], desc=row['descripcion'], marca=row['marca'],
                 cat=row['categoria'], precio=row['precio'],
                 stock=row['stock_actual'], futuro=row['stock_futuro'])

        # Crear relaciones Pedido -> Producto
        for _, row in detalle_op.iterrows():
            session.run("""
                MATCH (o:Pedido {id_pedido: $id_pedido})
                MATCH (pr:Producto {id_producto: $id_prod})
                MERGE (o)-[r:INCLUYE {nro_item: $item, cantidad: $cantidad}]->(pr)
            """, id_pedido=row['id_pedido'], id_prod=row['id_producto'],
                 item=row['nro_item'], cantidad=row['cantidad'])

    print("Datos cargados correctamente a Neo4j.")


cargar_datos_a_neo4j()
